In [1]:
%run init_notebookspace.py
from settings import DATA_DIR, MODEL_DIR, POST_EST_DIR

DATA_DIR is existant under: C:\Users\LukasGrahl\Documents\GIT\memoire1\data


In [2]:
%matplotlib inline

import arviz as az
from gEconpy.classes.model import gEconModel

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import xarray as xr
import pandas as pd

import os
import time
import math
from scipy.stats import percentileofscore
from copy import deepcopy

from scipy.stats import gamma, norm, beta, uniform
from filterpy.kalman import KalmanFilter
from filterpy.common import Saver


from src.plotting import plot_dfs
from src.process_data import load_data
from src.filtering_sampling import set_up_kalman_filter, kalman_filter, sample_from_priors, solve_updated_mod, get_Sigma
from src.utils import printProgBar, get_arr_pdf_from_dist
from src.classes import Spinner

from config import plt_config
plt.rcParams.update(plt_config)

load data

In [10]:
from config import fred_dict

df = load_data('prepro_data.csv', DATA_DIR, fred_dict)

# using real potential GDP instead of GDP
df = df.drop(['Ix', 'Zx', 'y', 'pi_s', 'w'], axis=1).rename(columns={'y_p': 'y', 'pi_c': 'pi'})
df = df.iloc[90:]

# split train and test
train = df[df['is_test'] == False].drop('is_test', axis=1).copy()
test = df[df['is_test'] == True].drop('is_test', axis=1).copy()

Error occured 's', file_dict may be incomplete
Error occured 'is_test', file_dict may be incomplete


load & solve model

In [5]:
from config import mod4_params, mod4_priors, mod5_params, mod5_priors, mod6_params, mod6_priors
mods = {
    'mod4_rbc_vanilla': {'params': mod4_params,
                         'priors': mod4_priors,
                         'is_lin': False,
                         'name': 'RBC'},
    'mod5_nk_vanilla_lin2': {'params': mod5_params,
                            'priors': mod5_priors,
                            'is_lin': True,
                             'name': 'NK'},
    'mod6_nk_energy_lin2': {'params': mod6_params,
                           'is_lin': True,
                           'priors': mod6_priors,
                           'name': 'NK Petrol'}
}


mod_name = 'mod5_nk_vanilla_lin2'
mod_is_linear = mods[mod_name]['is_lin']

mod = gEconModel(os.path.join(MODEL_DIR, f'{mod_name}.gcn'), verbose=False)
_, mod = solve_updated_mod(mod, verbose=True, model_is_linear=mod_is_linear)

mod_params = mod.free_param_dict
prior_dist = mods[mod_name]['priors']

Steady state found! Sum of squared residuals is 6.1286653075070395e-27
Solution found, sum of squared residuals:  2.563134175042647e-31
Norm of deterministic part: 0.000000000
Norm of stochastic part:    0.000000000
Model solution has 2 eigenvalues greater than one in modulus and 2 forward-looking variables.
Blanchard-Kahn condition is satisfied.


## drawing from priors

In [14]:
from scipy.stats import multivariate_normal

# number of draws for the sampler
n_runs = 20_000

verbose = False
start = time.time()

# counters
counter_solved = 0 # model was sovable
counter_kalman = 0 # kalman filter did not fail
counter_accp = 0 # draw was accepted into posterior

# reset params in the model
mod.free_param_dict.update(mod_params)

# get params, variables and shocks as lists
shock_names = [x.base_name for x in mod.shocks]
state_variables = [x.base_name for x in mod.variables]
model_params = list(mod.free_param_dict.keys())

# set kalman filter observed variables
observed_vars = ['y', 'n', 'r']
_ = [item for item in observed_vars if item not in state_variables]
assert len(_) == 0, f"{_} not in state variables"

# get posterior output list
param_posterio_list = {item: [mod.free_param_dict[item]] for item in model_params if item in prior_dist.keys()}
shock_posterior_list = {item: [.1] for item in shock_names}
loglike_list = [-100]

# get simga and scaling factor for the random walk law of motion
p, s = get_Sigma(prior_dist, mod_params, shock_names)
G_params = np.array(list(p.values())) 
G_shocks = np.array(list(s.values()))

# sacling factor for random walk law of motion
scaling_factor = .25

# get final ouput
output_dict = {}

for i in range(0, n_runs):
    printProgBar(i, n_runs-1, prefix='Progress')
    if (i== int(n_runs/3)) & (counter_solved>0):
        if (counter_accp/counter_solved <= .05):
            print(f'Current acceptance rate below 5%')
    
    
    # set dict to capture results of this run
    draw_dict = {
        'log_like_list': None,
        'log_like_sum': None,
        'is_solved': False,
        'ratio': None,
        'omega': None,
        'is_KF_solved': False,
        'KF_execption': None,
        'is_accepted': False,
        'parameters': {
            'prior': {item: None for item in model_params if item in prior_dist.keys()},
            'prior_pdf_p': {item: None for item in model_params if item in prior_dist.keys()},
            'posterior': {item: None for item in model_params if item in prior_dist.keys()}
        },
        'shocks': {
            'prior': {item: None for item in shock_names},
            'prior_pdf_s': {item: None for item in shock_names if item in prior_dist.keys()},
            'posterior': {item: None for item in shock_names}
        }
    }
    # current posterior
    old_posterior_p = {item: vals[-1] for item, vals in param_posterio_list.items()}
    old_posterior_s = {item: vals[-1] for item, vals in shock_posterior_list.items()}
    old_loglike = loglike_list[-1]
    
    # save posterior information to output dict
    draw_dict['parameters']['posterior'] = old_posterior_p
    draw_dict['shocks']['posterior'] = old_posterior_s
    
    
    # sample from priors according to random walk law of motion
    # a multivariate normal distribution wit G as standard deviation
    
    # prior for parameters
    prior = np.array(list(old_posterior_p.values()) + multivariate_normal(list([0] * len(old_posterior_p)), scaling_factor * G_params).rvs())
    
    # prior for shocks
    shocks = np.array(list(old_posterior_s.values()) + multivariate_normal(list([0] * len(old_posterior_s)), scaling_factor * G_shocks).rvs())
    
    # put priors into dictionary for further process
    prior, shocks = dict(zip(old_posterior_p.keys(), prior)), dict(zip(old_posterior_s.keys(), shocks))

    draw_dict['parameters']['prior'].update(prior)
    draw_dict['shocks']['prior'].update(shocks)
    
    # update model with new parameters and shocks
    mod.free_param_dict.update(prior)
    mod.shock_priors.update(shocks)

    # solve model for new steady state and transition matrix T
    # if model is not solved discard and proceed to nex iteration
    is_solved, mod = solve_updated_mod(mod, verbose=verbose, model_is_linear=mod_is_linear)
    if not is_solved:
        output_dict[i] = draw_dict
        continue
    else:
        draw_dict['is_solved'] = True
        counter_solved += 1
    
    # get Kalman filter matrices
    T, R = mod.T.values, mod.R.values
    H, Z, T, R, QN, zs = set_up_kalman_filter(R=R, T=T, observed_data=train[observed_vars].values, observed_vars=observed_vars, 
                                              shock_names=shock_names, shocks_drawn_prior=shocks, state_variables=state_variables)
       
    # set up Kalman filter
    kfilter = KalmanFilter(len(state_variables), len(observed_vars))
    kfilter.F = T
    kfilter.Q = QN
    kfilter.H = Z
    kfilter.R = H

    # run Kalman filter
    try:
        saver = Saver(kfilter)
        mu, cov, _, _ = kfilter.batch_filter(zs, saver=saver)
        ll = saver.log_likelihood
        
         # catch -math.inf values in log_likelihood, meaning that the filter did not converge
        if len([val for val in ll if val == -math.inf]) >0:
            output_dict[i] = draw_dict
            continue
        
        # sum-up individual log-likelihoods in order to obtain the ll for this run
        new_loglike = np.sum(ll)
        loglike_list.append(new_loglike)
    
        # save information in dict and update counters
        draw_dict['log_like_sum'] = new_loglike
        draw_dict['log_like_list'] = ll
        draw_dict['is_KF_solved'] = True
        counter_kalman += 1
    
    # catch possible errors in KFilter for debugging
    except Exception as e:
        draw_dict['KF_execption'] = e
        output_dict[i] = draw_dict
        continue
    
    # Metropolis Hastings MCMC sampler
    
    # get prior likelihood
    prior_pdf_p = get_arr_pdf_from_dist(prior, prior_dist)
    prior_pdf_s = get_arr_pdf_from_dist(shocks, prior_dist)
    prior_pdf = np.append(prior_pdf_p, prior_pdf_s)
    
    # save prior likelihood
    draw_dict['parameters']['prior_pdf_p'] = dict(zip(prior.keys(), prior_pdf_p))
    draw_dict['shocks']['prior_pdf_s'] = dict(zip(prior.keys(), prior_pdf_s))
    
    # get current posterior likelihood    
    mask_old_post = np.append(get_arr_pdf_from_dist(old_posterior_p, prior_dist), get_arr_pdf_from_dist(old_posterior_s, prior_dist))
    
    # compare current prior likelihood with the likelihood of the most recent draw accepted into the posterior
    ratio = (new_loglike * prior_pdf / old_loglike * mask_old_post).mean()
    ω = min([ratio, 1])
    # save ratios
    draw_dict['ratio'] = ratio
    draw_dict['omega'] = ω
              
    # MH sampler random acceptance, based on uniform distribution U(0,1)
    rand = np.random.uniform(0, 1)
    if rand <= ω:
        counter_accp += 1.
        draw_dict['is_accepted'] = True
        
        # update param posterior
        for key in prior.keys():
            param_posterio_list[key].append(prior[key])
        
        # update shock posterior
        for key in shock_posterior_list:
            shock_posterior_list[key].append(shocks[key])
                
    else:
        # leave posterior unaltered and restart
        is_accepted = False
        
    # save output    
    output_dict[i] = draw_dict

# print stats
print('\nloop ran for', (time.time() - start) / 60, 'minutes')
print('\nsolver rate', counter_solved/n_runs)
print('\nacceptance rate', counter_accp/counter_solved)

Progress |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 

loop ran for 4.84713431596756 minutes

solver rate 0.6332

acceptance rate 0.7792166771951989


In [15]:
params = list(param_posterio_list.keys())

xarr = xr.Dataset(
    {
        # draw & param dimension
        'posterior_param': (['draw', 'parameter'],[np.array(list(output_dict[i]['parameters']['posterior'].values())) for i in output_dict.keys()]),
        'posterior_shock': (['draw', 'shock'],[np.array(list(output_dict[i]['shocks']['posterior'].values())) for i in output_dict.keys()]),
        'prior_param': (['draw', 'parameter'], [np.array(list(output_dict[i]['parameters']['prior'].values())) for i in output_dict.keys()]),
        'prior_shock': (['draw', 'shock'], [np.array(list(output_dict[i]['shocks']['prior'].values())) for i in output_dict.keys()]),
        'prior_pdf_params': (['draw', 'parameter'], [np.array(list(output_dict[i]['parameters']['prior_pdf_p'].values())) for i in output_dict.keys()]),
        'prior_pdf_shocks': (['draw', 'shock'], [np.array(list(output_dict[i]['shocks']['prior_pdf_s'].values())) for i in output_dict.keys()]),
        
        # draw dimension
        'is_solved': (['draw'], [output_dict[i]['is_solved'] for i in output_dict.keys()]),
        'is_accepted': (['draw'], [output_dict[i]['is_accepted'] for i in output_dict.keys()]),
        'log_likelihood': (['draw'], [output_dict[i]['log_like_sum'] for i in output_dict.keys()]),
        'mh_ratio': (['draw'], [output_dict[i]['ratio'] for i in output_dict.keys()]),
        
        # uni dimensional
        'n_runs_acc': (['uni_dim'], [counter_accp]),
        'n_runs': (['uni_dim'], [n_runs]), # number of solved models 
        # 'kalman_obs_var': (['uni_dim'], (observed_vars)),
    },
    coords={
        'draw': (['draw'], list(range(0, int(n_runs)))),
        'parameter': (['parameter'], params),
        'shock': (['shock'], shock_names),
        'uni_dim': (['uni_dim'], [0])
    }
)

# only accepted runs
xarr_acc = xarr.where(xarr.draw >= int(xarr.n_runs / 2)).dropna('draw').copy()
# xarr_acc.where(xarr.is_accepted).dropna('draw')

# get quantiles of parameters
arr_nan = deepcopy(xarr_acc.posterior_param.values)
arr_nan = np.array(arr_nan, dtype=float)

for i in range(0, arr_nan.shape[1]):
    arr_no_nan = arr_nan[~np.isnan(arr_nan[:, i]), i]
    arr_nan[~np.isnan(arr_nan[:, i]), i] = [percentileofscore(arr_no_nan, x, 'rank') for x in arr_no_nan]
xarr_acc = xarr_acc.assign({'posterior_percentiles': (['draw', 'parameter'], arr_nan)})


quantile_ind = []
for i in [25, 50, 75]:
    arr = np.abs(xarr_acc.posterior_percentiles.mean(axis=1) - i)
    quantile_ind.append(np.where(arr==np.min(arr))[0][0])
    
xarr_acc = xarr_acc.assign({
    'posterior_q1': (['parameter'], xarr.sel(draw=quantile_ind[0]).posterior_param.values),
    'posterior_q2': (['parameter'], xarr.sel(draw=quantile_ind[1]).posterior_param.values),
    'posterior_q3': (['parameter'], xarr.sel(draw=quantile_ind[2]).posterior_param.values)
})

In [16]:
observed_vars

['y', 'n', 'r']

In [18]:
# save output
from datetime import datetime

a = ''.join(str(datetime.now().date()).split('-'))
b = ''.join((str(datetime.now().time()).split(':'))[:-1])
timestamp = '_'.join([a, b])

file_path = os.path.join(POST_EST_DIR, f'{mod_name}_{timestamp}.nc')
file_path_acc = os.path.join(POST_EST_DIR, f'{mod_name}_accepted_{timestamp}.nc')
print(file_path)
if not os.path.exists(file_path):
    xarr.to_netcdf(file_path)
    
if not os.path.exists(file_path_acc):
    xarr_acc.to_netcdf(file_path_acc)
    
else:
    print('File existst already')


C:\Users\LukasGrahl\Documents\GIT\memoire1\data\posterior_est_out\mod5_nk_vanilla_lin2_20230523_1405.nc
